## web spoffing

In [1]:
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup as BSoup
import requests

In [2]:
site = requests.get('https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00166.htm')
# soup = BSoup(site)
site

<Response [200]>

In [3]:
content = site.content

In [4]:
soup = BSoup(content, 'html.parser')

In [5]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<!-- [if lt IE 7]> <html class="ie6 oldie"> <![endif] -->
<!-- [if IE 7]>    <html class="ie7 oldie"> <![endif] -->
<!-- [if IE 8]>    <html class="ie8 oldie"> <![endif] -->
<!-- [if gt IE 8]> <! -->
<html class="">
 <!-- <![endif] -->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="vote_117_1_00166.xml" name="object"/>
  <meta content="2.0" name="version"/>
  <meta content="/Company Home/Sites/senategov/documentLibrary/Senate.gov/legislative/LIS/roll_call_votes/vote1171" name="path"/>
  <meta content="Friday, August 11, 2023" name="date"/>
  <meta content="8:06:51 AM EDT" name="time"/>
  <meta content="" name="keywords"/>
  <meta content="legislative" name="bucket"/>
  <meta content="" name="description"/>
  <title>
   U.

In [6]:
a = soup.findAll('div')

In [7]:

for x in a:
    try:
        print(x.get_attribute_list('class'))
        if 'newspaperDisplay_3column' == x['class']:
            print(x)
    except Exception as e:
        print('exception', e)

[None]
exception 'class'
['row']
['fluid']
['fluid']
['fluid']
['fluid', 'watch_text']
[None]
exception 'class'
['row']
[None]
exception 'class'
['fluid']
['nav']
[None]
exception 'class'
['row']
[None]
exception 'class'
[None]
exception 'class'
[None]
exception 'class'
[None]
exception 'class'
[None]
exception 'class'
['fluid', 'watch_text']
['fluid']
['fluid']
['contenttext']
[None]
exception 'class'
['contenttext']
['contenttext']
[None]
exception 'class'
['contenttext']
['contenttext']
[None]
exception 'class'
['contenttext']
['contenttext']
['contenttext']
['contenttext']
['contenttext']
[None]
exception 'class'
['contenttext']
[None]
exception 'class'
[None]
exception 'class'
['contenttext']
['contenttext']
[None]
exception 'class'
['contenttext']
['contenttext']
['contenttext']
['contenttext']
['contenttext']
['newspaperDisplay_3column']
['contenttext']
['contenttext']
['contenttext']
['contenttext']
['contenttext']
['contenttext']
['newspaperDisplay_3column']
['contenttext']
['

In [8]:
xml_file = open('vote_117_1_00166.xml', 'r').read()
xml_parser = BSoup(xml_file)

/home/anni/Data/BS/BS_venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [9]:
from bs4.element import Tag
count = 0
list_members = []
for x in xml_parser.members:
    count+= 1
    # print(type(x))
    # print(x)
    if type(x) == Tag:
        list_members.append(x)
print(len(list_members))
print(*list_members)

100
<member>
<member_full>Baldwin (D-WI)</member_full>
<last_name>Baldwin</last_name>
<first_name>Tammy</first_name>
<party>D</party>
<state>WI</state>
<vote_cast>Yea</vote_cast>
<lis_member_id>S354</lis_member_id>
</member> <member>
<member_full>Barrasso (R-WY)</member_full>
<last_name>Barrasso</last_name>
<first_name>John</first_name>
<party>R</party>
<state>WY</state>
<vote_cast>Yea</vote_cast>
<lis_member_id>S317</lis_member_id>
</member> <member>
<member_full>Bennet (D-CO)</member_full>
<last_name>Bennet</last_name>
<first_name>Michael</first_name>
<party>D</party>
<state>CO</state>
<vote_cast>Yea</vote_cast>
<lis_member_id>S330</lis_member_id>
</member> <member>
<member_full>Blackburn (R-TN)</member_full>
<last_name>Blackburn</last_name>
<first_name>Marsha</first_name>
<party>R</party>
<state>TN</state>
<vote_cast>Nay</vote_cast>
<lis_member_id>S396</lis_member_id>
</member> <member>
<member_full>Blumenthal (D-CT)</member_full>
<last_name>Blumenthal</last_name>
<first_name>Richar

### Creating dataframe from xml file

In [10]:
def get_member_details(member:Tag):
    name = member.first_name.string + " " + member.last_name.string
    party = member.party.string
    state = member.state.string
    vote_caste = member.vote_cast.string
    lis_member_id = member.lis_member_id.string
    return [name, party, state, vote_caste, lis_member_id]

In [11]:
list_data = list()
list_of_header = ['Name', 'Party', 'State', 'Vote_caste', 'lis_member_id']
list_data.append(list_of_header)
for member in list_members:
    member_data = get_member_details(member)
    list_data.append(member_data)

# print(*list_data, sep='\n')
    



### Writing into CSV file


In [12]:
# for m in list_data:
import csv
f = open('senate_filtered_data.csv', 'w')
csv_writer = csv.writer(f)
csv_writer.writerows(list_data)
f.close()
    

In [13]:
senate_pd = pd.read_csv('senate_filtered_data.csv')

In [14]:
senate_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           100 non-null    object
 1   Party          100 non-null    object
 2   State          100 non-null    object
 3   Vote_caste     100 non-null    object
 4   lis_member_id  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


### **Question: 14:** How many senators did not vote?

In [15]:
print('Number of senates that did not vote is:', len(senate_pd[senate_pd['Vote_caste'] == 'Not Voting']) )

Number of senates that did not vote is: 5


### **Question: 15:** What is the ratio of number of rows to the number of columns in your dataset?

In [16]:
print('Unique vote values is:', senate_pd['Vote_caste'].unique())

Unique vote values is: ['Yea' 'Nay' 'Not Voting']


In [17]:
print('Ratio between number of Rows and Number of Columns is:', (len(senate_pd)/ len(senate_pd.columns)))

Ratio between number of Rows and Number of Columns is: 20.0


### **Question 1:** How many Democrats did not vote?

In [18]:
answer_1 = senate_pd[(senate_pd['Party'] == 'D') & (senate_pd['Vote_caste'] == 'Not Voting')]
print('Number of democrats did not vote is:', len(answer_1))
print('Members are:',  answer_1, sep='\n')


Number of democrats did not vote is: 2
Members are:
            Name Party State  Vote_caste lis_member_id
13    Ben Cardin     D    MD  Not Voting          S308
63  Patty Murray     D    WA  Not Voting          S229


### **Question 4:** How many states voted unanimously with 'yes'?

In [19]:
yes_count = 0
list_state = []
for x in senate_pd['State'].unique():
    # print(senate_pd[senate_pd['State'] == x])
    is_unanimously_voted_yes = True
    val = senate_pd[senate_pd['State'] == x]
    # print(type(val))
    for index, row in val.iterrows():
        # print(row)
        # print('Vote caste for current state {0} is: {1}'.format(x, row['Vote_caste']))
        # print('type:', type(row['Vote_caste']))
        if row['Vote_caste'] != 'Yea':
            is_unanimously_voted_yes = False
    if is_unanimously_voted_yes:
        yes_count += 1
        list_state.append(x)

In [20]:
print(list_state, yes_count)

['WI', 'WY', 'CO', 'CT', 'NJ', 'OH', 'NC', 'WV', 'DE', 'PA', 'ME', 'NV', 'MT', 'IL', 'IA', 'CA', 'NE', 'NY', 'NH', 'NM', 'HI', 'MS', 'OK', 'VA', 'AZ', 'MN', 'VT', 'MA', 'KS', 'OR', 'GA', 'MI', 'RI'] 33


In [21]:
print('33 states votes yea ')

33 states votes yea 


### **Question 11:** How many Democrats voted 'yea'?

In [22]:
democrats_pd = senate_pd[(senate_pd['Party'] == 'D') & (senate_pd['Vote_caste'] == 'Yea') ]

In [23]:
print(' Total number Democrate voted "Yea" is: ',len(democrats_pd))

 Total number Democrate voted "Yea" is:  46


### **Question 2** How many Republicans voted 'yea'?

In [24]:
republicans_pd = senate_pd[(senate_pd['Party'] == 'R') & (senate_pd['Vote_caste'] == 'Yea')]

In [25]:
print('Total number of Republican voted "Yea" is:', len(republicans_pd))

Total number of Republican voted "Yea" is: 34


### **Question 9** What is the ratio of Democrats' major decision votes to Republicans' major decision votes?

In [26]:
democrats_pd = senate_pd[senate_pd['Party'] == 'D']
republicans_pd = senate_pd[senate_pd['Party'] == 'R']

In [27]:
def return_majority_val(data:pd.DataFrame):
    yea_count, nay_count, nan_count = (0,0,0)
    for index, row in data.iterrows():
        if row['Vote_caste'] == 'Yea':
            yea_count += 1
        elif row['Vote_caste'] =='Nay':
            nay_count += 1
        else:
            nan_count += 1
    print('counts are ', yea_count, ' ',  nay_count, ' ', nan_count)
    democratic_majorty = 'Yea' if yes_count > nay_count and yes_count > nan_count else 'Nay' if nay_count > yes_count and nay_count > nan_count else 'Not Voting'
    majorty_count = max([yea_count, nay_count, nan_count])

    return [democratic_majorty, majorty_count]

In [28]:
# democrats_pd
# computing major decision of democrats with number of candidates
#  ['Yea' 'Nay' 'Not Voting']
democratic_majorty, dem_majorty_count = return_majority_val(democrats_pd)
print('Majorty of democratics is:', democratic_majorty, ' with count', dem_majorty_count)


counts are  46   0   2
Majorty of democratics is: Yea  with count 46


In [29]:
republic_majority, rep_majority_count = return_majority_val(republicans_pd)
print('Majority of Republican is:', republic_majority, ' ', rep_majority_count)

counts are  34   13   3
Majority of Republican is: Yea   34


In [30]:
ratio_of_majority_count = dem_majorty_count / rep_majority_count
print('Ratio between Democrats and Republican is:', ratio_of_majority_count)

Ratio between Democrats and Republican is: 1.3529411764705883


### **Question 3** What is the total number of 'Yea' votes from senators whos names start with 'B'?

In [51]:
senate_pd.info()
senate_pd.head(100)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           100 non-null    object
 1   Party          100 non-null    object
 2   State          100 non-null    object
 3   Vote_caste     100 non-null    object
 4   lis_member_id  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


,Name,Party,State,Vote_caste,lis_member_id
0,Tammy Baldwin,D,WI,Yea,S354
1,John Barrasso,R,WY,Yea,S317
2,Michael Bennet,D,CO,Yea,S330
3,Marsha Blackburn,R,TN,Nay,S396
4,Richard Blumenthal,D,CT,Yea,S341
...,...,...,...,...,...
95,Elizabeth Warren,D,MA,Yea,S366
96,Sheldon Whitehouse,D,RI,Yea,S316
97,Roger Wicker,R,MS,Yea,S318
98,Ron Wyden,D,OR,Yea,S247


In [47]:
member_with_yea_and_b_count = 0
senate_with_yes = senate_pd

for index, member in senate_with_yes.iterrows():
    if member['Name'][0] == 'B' or member['Name'] == 'b':
        print('name of the player is: \'', member['Name'], '\' their vote:', member['Vote_caste'], 'Party is:', member['Party'])
        member_with_yea_and_b_count += 1

print('Sentate with Vote "yea" and name starting with B is:', member_with_yea_and_b_count)


name of the player is: ' Ben Cardin ' their vote: Not Voting Party is: D
name of the player is: ' Bob Casey ' their vote: Yea Party is: D
name of the player is: ' Bill Cassidy ' their vote: Yea Party is: R
name of the player is: ' Bill Hagerty ' their vote: Yea Party is: R
name of the player is: ' Ben Lujan ' their vote: Yea Party is: D
name of the player is: ' Bernard Sanders ' their vote: Yea Party is: I
name of the player is: ' Ben Sasse ' their vote: Yea Party is: R
name of the player is: ' Brian Schatz ' their vote: Yea Party is: D
Sentate with Vote "yea" and name starting with B is: 8


### **Question 8** How many senators have names starting with 'W'?

In [49]:
senate_with_w_name_count = 0

for index, member in senate_pd.iterrows():
    if member.Name[0] == 'W':
        print('Name with W alphabet is:', member.Name)
        senate_with_w_name_count += 1

print('Total number of senate count with W alphabet is:', senate_with_w_name_count)

Total number of senate count with W alphabet is: 0


In [33]:
print()
for i, x in senate_pd.sort_values(['Name']).iterrows():
    print(x.Name) 


Alex Padilla
Amy Klobuchar
Angus King
Ben Cardin
Ben Lujan
Ben Sasse
Bernard Sanders
Bill Cassidy
Bill Hagerty
Bob Casey
Brian Schatz
Catherine Cortez Masto
Charles Grassley
Charles Schumer
Chris Van Hollen
Christopher Coons
Christopher Murphy
Cindy Hyde-Smith
Cory Booker
Cynthia Lummis
Dan Sullivan
Deb Fischer
Debbie Stabenow
Dianne Feinstein
Edward Markey
Elizabeth Warren
Gary Peters
Jacklyn Rosen
James  Risch
James Lankford
Jeanne Shaheen
Jeff Merkley
Jerry Moran
Jim Inhofe
John Barrasso
John Boozman
John Cornyn
John Hickenlooper
John Hoeven
John Kennedy
John Reed
John Thune
Jon Ossoff
Jon Tester
Joni Ernst
Joseph Manchin
Josh Hawley
Kevin Cramer
Kirsten Gillibrand
Kyrsten Sinema
Lindsey Graham
Lisa Murkowski
Maggie Hassan
Marco Rubio
Maria Cantwell
Mark Kelly
Mark Warner
Marsha Blackburn
Martin Heinrich
Mazie Hirono
Michael Bennet
Mike Braun
Mike Crapo
Mike Lee
Mike Rounds
Mitch McConnell
Mitt Romney
Patrick Leahy
Patrick Toomey
Patty Murray
Rand Paul
Raphael Warnock
Richard Blume

### **Question 5** How many states had at least one non-voting senator?


In [34]:
#  ['Yea' 'Nay' 'Not Voting']
state_non_voting_senate_count = 0
for state in senate_pd['State'].unique():
    for i, senate in senate_pd[senate_pd['State'] == state].iterrows():
        if senate['Vote_caste'] == 'Not Voting':
            print('atleast non voting state is:', state)
            state_non_voting_senate_count += 1
            break # for preventing recounting
print('Total states with atleast one Not voting member is:', state_non_voting_senate_count)

atleast non voting state is: WA
atleast non voting state is: MD
atleast non voting state is: ND
atleast non voting state is: KY
atleast non voting state is: SD
Total states with atleast one Not voting member is: 5


## Question 15 is incorrect
## Question 3 is incorrect
## Question 8 is incorrect